Definindo o atraso da maré através da correlação entre a série original e os dados observados.

A partir dessa diferença de fase, poderemos redefinir a fase das componentes de maré, baseado no período de cada componente.



In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import os
import string
import xray as xr
from scipy.spatial import cKDTree

import matplotlib
matplotlib.style.use('ggplot')

import sys
sys.path.append('../rotinas/artigoTGpack/')

import artigoTGpack as oceano

%matplotlib notebook

/home/danilo/miniconda2/envs/mestrado/lib/python2.7/site-packages/xarray/conventions.py:9: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


In [2]:
# dados de saída do modelo já determinados para um período e dados do BNDO
# para o mesmo período
secom = np.asarray([
        0.05066382, -0.14191973,  0.14863421, -0.07172287,  0.13184801,
       -0.18373264,  0.12726995, -0.1397833 ,  0.13581566, -0.1556539 ,
        0.1226919 , -0.14924462,  0.12147108, -0.04486495,  0.09308713,
       -0.12818556,  0.02655272,  0.02929956,  0.03052037, -0.06500839,
       -0.08393102,  0.12757516, -0.02624752, -0.00366244, -0.20845414,
        0.22005188, -0.07019686,  0.09522356, -0.30276209,  0.28506028,
       -0.10468488,  0.17976499, -0.39035556,  0.31222341, -0.14405616,
        0.23927972, -0.44803906,  0.30642454, -0.15840073,  0.29696322,
       -0.45170151,  0.27743018, -0.16175797,  0.31466504, -0.41355104,
        0.2215779 , -0.14344575,  0.295132  , -0.33023043,  0.14924462,
       -0.10712651,  0.23287044, -0.21638944,  0.06287197, -0.04791698,
        0.13734167, -0.08637265, -0.02929956,  0.02044865,  0.02136426,
        0.04455974, -0.10804212,  0.06928124, -0.12025027,  0.12360751,
       -0.15046543,  0.1397833 , -0.19777201,  0.17640775, -0.13886769,
        0.16694644, -0.21425301,  0.16603082, -0.07477491,  0.14405616,
       -0.20296047,  0.04517015,  0.02258508,  0.07965817, -0.11689303,
       -0.09369754,  0.14130932,  0.00122081,  0.01495498, -0.24507859,
        0.24935144, -0.09583397,  0.17030368, -0.34335419,  0.33389287,
       -0.16755684,  0.25942316, -0.46818251,  0.36532885, -0.20936975,
        0.33725011, -0.49565084,  0.3448802 , -0.21486342,  0.35678315,
       -0.46390966,  0.27651457, -0.18739508,  0.33511369, -0.36777048,
        0.18556386, -0.13001679,  0.25637113, -0.23897451,  0.08484663,
       -0.05951473,  0.15229666, -0.10193804, -0.01068213,  0.00518846,
        0.03357241,  0.02380589, -0.07324889])

bndo = np.asarray([-0.34568428,  0.08764906, -0.11901761,  0.26764906, -0.16568428,
        0.28431572,  0.11431572, -0.16235094, -0.01568428,  0.15431572,
       -0.22901761,  0.11764906,  0.08431572,  0.04431572,  0.14431572,
       -0.13901761,  0.38098239,  0.08098239,  0.14098239, -0.25901761,
        0.34431572, -0.22235094,  0.13098239, -0.56568428,  0.54431572,
       -0.19901761,  0.47431572, -0.56901761,  0.62764906, -0.28901761,
        0.60431572, -0.52901761,  0.73431572, -0.23901761,  0.62764906,
       -0.48235094,  0.45764906, -0.21568428,  0.60431572, -0.43235094,
        0.57431572, -0.04901761,  0.38764906, -0.16235094,  0.17431572,
        0.05764906,  0.57431572, -0.10568428,  0.25098239,  0.17764906,
        0.29431572, -0.00901761,  0.02098239,  0.23431572,  0.20098239,
        0.19431572,  0.06431572,  0.38098239,  0.23764906,  0.37431572,
        0.06431572,  0.33431572, -0.05568428,  0.35098239,  0.00764906,
        0.40764906, -0.01901761,  0.40764906, -0.00568428,  0.13098239,
       -0.16235094,  0.05764906, -0.07235094,  0.04431572, -0.21901761,
        0.03098239,  0.07098239, -0.04235094,  0.06098239, -0.18235094,
        0.34764906, -0.29901761,  0.15764906, -0.71568428,  0.37098239,
       -0.55901761,  0.16431572, -0.98235094,  0.40431572, -0.47235094,
        0.42098239, -0.81235094,  0.62764906, -0.28901761,  0.61764906,
       -0.66235094,  0.65431572, -0.08901761,  0.63431572, -0.39568428,
        0.33098239,  0.00431572,  0.23764906, -0.27568428, -0.22568428,
       -0.11568428,  0.12431572, -0.28901761, -0.31235094,  0.09098239,
        0.21431572,  0.06431572, -0.22568428, -0.17568428, -0.20901761,
       -0.06235094, -0.35568428,  0.18098239])

time = pd.date_range(start='1996-12-18 03:00', end='1997-01-16 09:00', freq='6H')

time.shape, bndo.shape, secom.shape

((118,), (118,), (118,))

In [3]:
df = pd.DataFrame({'secom':secom, 'bndo':bndo},index=time)

df.corr()

,bndo,secom
bndo,1.000000,-0.655862
secom,-0.655862,1.000000


In [4]:
oceano.skill_willmott(df.bndo.values, df.secom.values)

0.065644770071519365

# Definindo a diferença de fase entre as séries

Source: https://stackoverflow.com/questions/4688715/find-time-shift-between-two-similar-waveforms

In [5]:
from scipy import signal, fftpack
import scipy

In [6]:
secomF = fftpack.fft(df.secom.values)
bndoF  = fftpack.fft(df.bndo.values)

c = fftpack.ifft(secomF * scipy.conj(bndoF))

phase_difference = np.argmax(np.abs(c))

phase_difference # in hours

NameError: name 'df' is not defined

In [3]:
def max_correlacao(modelo, observado):
    '''
    obtem a diferenca temporal (em horas) da maxima correlação entre
    duas séries temporais.
    modelo: série temporal saída do modelo, em pandas.Series
    observado: série temporal obtida in situ, em pandas.Series
    
    return
        diferença de fase em horas
    '''
    z = signal.fftconvolve(modelo, observado[::-1])
    lags = np.arange(z.size) - (observado.size - 1)
    return ( lags[np.argmax(np.abs(z))] )

lag = max_correlacao(df.secom.values, df.bndo.values) + 24

NameError: name 'df' is not defined

# Realizando a alteração nas componentes de maré

In [7]:
BASE_DIR = oceano.make_dir()
DATA_DIR = BASE_DIR + 'artigoTG/rotinas/tides/'

DATA_DIR

'/media/danilo/Danilo/mestrado/github/artigoTG/rotinas/tides/'

In [12]:
lag = 11 # horas

# período das componentes de maré aceitas no sECOM
Tcomponentes = [12., 12.42, 12.66, 23.94, 24.06, 25.86]

# BASE_DIR = '/home/tparente/danilo/mestrado/github/artigoTG/'
# DATA_DIR = BASE_DIR + 'rotinas/tides/'
# DATA_DIR = '/media/danilo/Danilo/mestrado/github/artigoTG/rotinas/tides/'

# leitura do arquivo com dados de maré (do run_data)
f = open(DATA_DIR+'tide_complement.txt')
f = f.read()
f = f.split('\n')
data = f[2:]

# inicializar novo arquivo de texto
out = open(DATA_DIR+'lag_11h_fromOriginal','w')
out.write(f[0]+'\n')
out.write(f[1]+'\n')

# loop 
for i in np.arange(0,len(data)-1,3):
    loc = data[i]
    com = data[i+1]
    deg = data[i+2]
    
    nova_fase  = []
    componente = 0
    
    for fase in string.split(deg,sep=' '):
        if len(fase)!=0:
            nfase = (lag/Tcomponentes[componente]) * 360
            nova_fase.append(float(nfase))
            componente += 1
            
    # imprimir em novo arquivo os dados modificados
    out.write(loc+'\n')
    out.write(com+'\n')
    out.write('%10.5f%10.5f%10.5f%10.5f%10.5f%10.5f\n'%(nova_fase[0],nova_fase[1],nova_fase[2],nova_fase[3],nova_fase[4],nova_fase[5]))
    
# fechar arquivo
out.close()

In [40]:
string.split(deg,sep=' ')

['',
 '176.70958',
 '167.28714',
 '206.56158',
 '188.78208',
 '176.35280',
 '125.76240']